In [1]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
from scipy.optimize import fsolve
from ipywidgets import interact, FloatSlider

# Import data of the VLE
# Add your path to the CSV file
file_path = 'VLE.csv'

# Read the CSV file into a DataFrame, change separator if necessary
df = pd.read_csv(file_path,sep=';')

# Replace 'x_data_column_name' and 'y_data_column_name' with your actual column names
x_data_column_name = 'x_data'  # Replace with the actual column name for x_data
y_data_column_name = 'y_data'  # Replace with the actual column name for y_data

# Convert the columns to numpy arrays
x_data = np.array(df[x_data_column_name])
y_data = np.array(df[y_data_column_name])

def pseudo_eq_wrapper(wd, wf, wb, nm, r_factor, q):
    pseudo_eq(wd, wf, wb, nm, r_factor, q, x_data, y_data)

def pseudo_eq(wd,wf,wb,nm,r_factor,q,x_data,y_data): 

    #convert weight percent in molar fraction
    xd=((wd*17)/32.05)/(((wd*17)/32.05)+(((1-wd)*17)/18.02))
    xf=((wf*17)/32.05)/(((wf*17)/32.05)+(((1-wf)*17)/18.02))
    xb=((wb*17)/32.05)/(((wb*17)/32.05)+(((1-wb)*17)/18.02))
    
    # Calcule the pseudo eq curve 
    y_nm = x_data+ (y_data-x_data)*nm

    # Perform polynomial regression for equilibrium curve
    degree = 10
    coefficients = np.polyfit(x_data, y_data, degree)
    poly_function = np.poly1d(coefficients)

    # Perform polynomial regression for pseudo eq curve
    coefficients2 = np.polyfit(x_data, y_nm, degree)
    poly_function2 = np.poly1d(coefficients2)

    # Define the inverse function using numerical methods
    def inverse_function(y, poly_func, x0):
        return fsolve(lambda x: poly_func(x) - y, x0)[0]

    # Equilibrium curve using the fitted polynomial
    def equilibrium_curve(x):
        return poly_function(x)

    def pseudoeq_curve(x):
        return poly_function2(x)
    
    # Inverse of the pseudo equilibrium curve
    def inverse_equilibrium_curve(y):
        return inverse_function(y, poly_function2, 0.5)

    # Rectifying section
    def rect(R,xD,x_rect):
            y_rect = R/(R+1)*x_rect + xD/(R+1)
            return y_rect

    # Stripping section
    def stp(xiF,yiF,xW,x_stp):  
        y_stp = ((yiF-xW)/(xiF-xW))*(x_stp-xW) + xW
        return y_stp

    #Calculate Rmin
    m=(xd-pseudoeq_curve(xf))/(xd-xf)
    rmin=m/(1-m)

    #Calculate R
    r = r_factor*rmin # Reflux ratio
    
    #Calculate yiF as function of q
    if q==1:
        xiF=xf
    else:
        xiF = (xf/(q-1)+xd/(r+1))/(q/(q-1)-r/(r+1))
    
    yiF = r/(r+1)*xiF + xd/(r+1)

    # Generate points for equilibrium curve
    x = np.linspace(0, 1, 100)
    y_eq = equilibrium_curve(x)

    # Generate points for equilibrium curve
    y_pseudo = pseudoeq_curve(x)

    # Operating lines
    # Rectifying
    x_rect = np.linspace(xiF,1,100)    
    y_rect = rect(r, xd,x_rect)

    # Stripping
    x_stp = np.linspace(0,xiF,100)  
    y_stp = stp(xiF,yiF,xb,x_stp)

    # Stages calculation
    x_stage = [xd]
    y_stage = [xd]  # Start at the distillate composition on the y=x curve

    current_x = xd
    current_y = xd
    num_stages=-1 #initate number of stages
    stages_list=[]
    mol_frac=[]

    while current_x > xb:
        if current_x > xiF:
            # Move horizontally to rectifying line
            current_y = rect(r,xd,current_x)
            x_stage.append(current_x)
            y_stage.append(current_y)
            num_stages+=1
            mol_frac.append(current_x)
            stages_list.append(num_stages)

        # Move vertically to equilibrium curve
        current_x = inverse_equilibrium_curve(current_y)  # Using inverse of equilibrium curve
        x_stage.append(current_x)
        y_stage.append(current_y)

        if current_x <= xf:
            # Move horizontally to stripping line
            current_y = stp(xiF, yiF, xb, current_x)
            x_stage.append(current_x)
            y_stage.append(current_y)
            num_stages +=1
            mol_frac.append(current_x)
            stages_list.append(num_stages)


    # Plotting
    plt.figure(figsize=(10, 8))

    # xb,xf,xd points
    plt.plot(xf,xf,'go',xd,xd,'go',xb,xb,'go',markersize=5)
    plt.text(xf+0.05,xf-0.03,'($x_{F}, x_{F}$)',horizontalalignment='center')
    plt.text(xd+0.05,xd-0.03,'($x_{D}, x_{D}$)',horizontalalignment='center')
    plt.text(xb+0.05,xb-0.03,'($x_{B}, x_{B}$)',horizontalalignment='center')

    # Feed line
    plt.plot(xiF,yiF,'go',markersize=5)
    plt.plot([xf,xiF],[xf,yiF],'k:')

    # Pseudo-eq and Equilibrium curve
    plt.plot(x, y_eq,'-',color='#1E2761',label='Equilibrium Curve')
    plt.plot(x, y_pseudo,'-',color= 'green', label='Pseudo Equilibrium Curve')

    # Intersection: Rectifying + Stripping + stages
    plt.plot(x,x,'k-')
    plt.plot(x_rect, y_rect, 'k--', label='Rectifying Line')
    plt.plot(x_stp, y_stp, 'k-.', label='Stripping Line')
    plt.plot(x_stage, y_stage, '-',color='#990011', label='Stages')

    # Creating a nice output boxes 
    textstr1 = '\n'.join((
            r'$Input:$',
            r'$q=%.2f$' % (q, ),
            r'$x_F=%.2f$' % (xf, ),
            r'$x_D=%.2f$' % (xd, ),
            r'$x_B=%.2f$' % (xb, ),
            r'$n_m=%.2f$' % (nm, )))
    textstr2 = '\n'.join((
            r'$Output:$',
            r'$R_{min}=%.2f$' % (rmin, ),
            r'$R=%.2f$' % (r, ),
            r'$Stages=%.0f$' % (num_stages, )))
    
    # place a text box in upper left in axes coords
    props = dict(boxstyle='round', facecolor='lightblue', alpha=0.5)
    plt.text(0.65, 0.2, textstr1, fontsize=10, verticalalignment='bottom', bbox=props)
    plt.text(0.65, 0.0, textstr2, fontsize=10, verticalalignment='bottom', bbox=props)

    # Add legends
    plt.title('Pseudo-Equilibrium Curve Methanol-Water system')
    plt.xlabel('x (-)')
    plt.ylabel('y (-)')
    plt.legend()
    plt.grid(False)
    plt.show()

    
    # Create a DataFrame from the x_data and y_data arrays
    frac = {
        'Molar fraction': mol_frac,
        'Stages': stages_list
    }
    frac_df = pd.DataFrame(frac)

    display(frac_df)

# Run this in jupyter notebook to change parameters
 
interact(pseudo_eq_wrapper,
wd=FloatSlider(value=0.99, min=0, max=1, step=0.01, description='wd'),
wf=FloatSlider(value=0.5, min=0, max=1, step=0.01, description='wf'),
wb=FloatSlider(value=0.1, min=0, max=1, step=0.01, description='wb'),
r_factor=FloatSlider(value=9.245, min=1, max=1000, step=0.001, description='r_factor'),
q=FloatSlider(value=1, min=0, max=1, step=0.05, description='q'),
nm=FloatSlider(value=0.784, min=0, max=1, step=0.001, description='nm'))


interactive(children=(FloatSlider(value=0.99, description='wd', max=1.0, step=0.01), FloatSlider(value=0.5, de…

<function __main__.pseudo_eq_wrapper(wd, wf, wb, nm, r_factor, q)>